In [ ]:
import os
import numpy as np 
import cv2
import torch
import torch.nn as nn
from torchvision import models
from torch.autograd import Variable
from sklearn.decomposition import PCA
from sklearn import svm
from sklearn.model_selection import StratifiedKFold  

In [ ]:
def read_images(path, folders):
    images = []
    labels = []
    idx = 0
    for folder in folders:
        for filename in os.listdir(path+folder):
            image = cv2.imread(os.path.join(path+folder, filename))
            if image is not None:
                images.append(image)
                labels.append(idx)
                
        idx += 1
    images = np.array(images)
    labels = np.array(labels)
    return images, labels

def getDescriptors(images, labels, model) : 
    features = []
    image_labels = []
    i = 0

    for image in images : 
        # Re-sizing the image
        image = cv2.normalize(image, None, alpha=0, beta=1, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_32F)
        if (image.shape[2] == 1):
            image = cv2.cvtColor(image,cv2.COLOR_GRAY2RGB)
        image = cv2.resize(image, (224, 224), interpolation=cv2.INTER_AREA)
        image = image.transpose(2, 1, 0)
        image = Variable(torch.Tensor(image))
        image = image.unsqueeze(0)
        feature = model(image)
        feature = feature.data.numpy().reshape(1000)
        if feature is not None : 
            features.append(feature)
            image_labels.append(labels[i])
        i += 1
        print (feature)

    features = np.asarray(features)
    return features, image_labels

In [ ]:
model = models.alexnet(pretrained=True)
new_classifier = nn.Sequential(*list(model.classifier.children()))
model.classifier = new_classifier
model.eval()

In [ ]:
root = "directory/101_ObjectCategories/"

set_images = [ item for item in os.listdir(root) if os.path.isdir(os.path.join(root, item)) ]
images, labels = read_images(root, set_images)

In [ ]:
features, image_labels = getDescriptors(images, labels, model)

In [ ]:
pca = PCA(n_components = 0.9) 
features = pca.fit_transform(features) 
print(features)

In [ ]:
features = np.asarray(features)
image_labels = np.asarray(image_labels)

md = svm.SVC(kernel='linear')
cv = StratifiedKFold(n_splits=6, random_state=42)

scores = []
count = 0
for train_ind, validate_ind in cv.split(features, image_labels):
    print("CV # - ", count)
    count += 1
    train_X, train_y = X[train_ind], y[train_ind]
    validate_X, validate_y = X[validate_ind], y[validate_ind]
    md.fit(train_X, train_y)
    score = md.score(validate_X, validate_y)
    scores.append(score)

sc = np.array(scores)
print(sc)
print("Score: " + str(np.mean(sc)))